## Imports

In [ ]:
import os
import random
import glob
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import numpy as np
import matplotlib.pyplot as plt

## Set random seed for reproducibility

In [ ]:
SEED = 42
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)

## Data Loading

In [ ]:
from sklearn.model_selection import train_test_split
from collections import Counter


data_dir = "../data/balanced_data"


transform = transforms.Compose([
    transforms.Resize((128, 128)),  
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  
])


dataset = datasets.ImageFolder(root=data_dir, transform=transform)
labels = [label for _, label in dataset.samples]
filenames = [os.path.basename(path) for path, _ in dataset.samples]


label_1_indices = [i for i, label in enumerate(labels) if label == 1]
label_0_indices = [i for i, label in enumerate(labels) if label == 0]


original_label_1_indices = [i for i in label_1_indices if filenames[i].startswith("original")]


label_1_test_size = int(0.1 * len(label_1_indices))  
label_1_test_indices = original_label_1_indices[:label_1_test_size]
remaining_label_1_indices = list(set(label_1_indices) - set(label_1_test_indices))


label_0_test_size = int(0.1 * len(label_0_indices))  
label_0_test_indices = label_0_indices[:label_0_test_size]
remaining_label_0_indices = label_0_indices[label_0_test_size:]


label_1_train_indices, label_1_val_indices = train_test_split(
    remaining_label_1_indices, test_size=0.2, stratify=[labels[i] for i in remaining_label_1_indices], random_state=42
)


label_0_train_indices, label_0_val_indices = train_test_split(
    remaining_label_0_indices, test_size=0.2, stratify=[labels[i] for i in remaining_label_0_indices], random_state=42
)


train_indices = label_1_train_indices + label_0_train_indices
val_indices = label_1_val_indices + label_0_val_indices
test_indices = label_1_test_indices + label_0_test_indices


train_dataset = torch.utils.data.Subset(dataset, train_indices)
val_dataset = torch.utils.data.Subset(dataset, val_indices)
test_dataset = torch.utils.data.Subset(dataset, test_indices)


batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


train_counts = Counter([labels[i] for i in train_indices])
val_counts = Counter([labels[i] for i in val_indices])
test_counts = Counter([labels[i] for i in test_indices])

print(f"Train class distribution: {train_counts}")
print(f"Validation class distribution: {val_counts}")
print(f"Test class distribution: {test_counts}")


## Architecture Definition

In [ ]:

class ConformerTinyBinary(nn.Module):
    def __init__(self, img_size=128, num_classes=5, embed_dim=32, num_heads=4, num_transformer_layers=1):
        super(ConformerTinyBinary, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, embed_dim, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(embed_dim),
            nn.ReLU(),
            nn.Conv2d(embed_dim, embed_dim, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(embed_dim),
            nn.ReLU(),
            nn.Conv2d(embed_dim, embed_dim, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(embed_dim),
            nn.ReLU()
        )
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads),
            num_layers=num_transformer_layers
        )
        self.fc = nn.Linear(embed_dim, num_classes)

    def forward(self, x):
        x = self.conv(x)
        b, c, h, w = x.shape
        x = x.view(b, c, -1).permute(2, 0, 1)  
        x = self.transformer(x)
        x = x.mean(dim=0)  
        x = self.fc(x)
        return x

## Function Definition

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc, classification_report
from scipy import interp

model_dir = 'scratch_augmentless/saved_models'
os.makedirs(model_dir, exist_ok=True)


def save_model(epoch, model, optimizer, path, best=False):
    state = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }
    torch.save(state, path)
    if best:
        best_path = os.path.join(model_dir, f"best_model_epoch{epoch}.pth")
        torch.save(state, best_path)

def load_model(path, model, optimizer):
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    return checkpoint['epoch']


def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
    return running_loss / len(dataloader.dataset)


def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels).item()
    return running_loss / len(dataloader.dataset), correct / len(dataloader.dataset)

def custom_metric(y_true, y_pred, min_tpr=0.8):
    """
    Calculate the partial AUC (pAUC) based on a minimum TPR threshold.

    Args:
        y_true (array): True binary labels.
        y_pred (array): Predicted probabilities.
        min_tpr (float): Minimum TPR threshold (default: 0.8).

    Returns:
        float: Scaled pAUC value.
    """
    
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    max_fpr = 1 - min_tpr  
    v_gt = abs(y_true - 1)  
    v_pred = 1.0 - y_pred  

    
    pauc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)

    
    pauc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (pauc_scaled - 0.5)

    return pauc



def compute_auc_and_roc(model, dataloader, device, min_tpr=0.8):
    """
    Computes the ROC, AUC, and partial AUC for the given model and dataloader.
    
    Args:
        model: The PyTorch model to evaluate.
        dataloader: DataLoader for evaluation data.
        device: Device to run the computations on (CPU or GPU).
        min_tpr: Minimum TPR threshold for partial AUC computation.
    
    Returns:
        fpr: False positive rates.
        tpr: True positive rates.
        roc_auc: Full area under the ROC curve.
        partial_auc: Partial area under the ROC curve for TPR >= min_tpr.
        y_true: Ground truth labels.
        y_scores: Predicted scores for the positive class.
    """
    model.eval()
    y_true = []
    y_scores = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            
            probabilities = torch.softmax(outputs, dim=1)[:, 1]
            
            y_true.extend(labels.cpu().numpy())
            y_scores.extend(probabilities.cpu().numpy())
    
    
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    roc_auc = roc_auc_score(y_true, y_scores)  

    
    normalized_pauc = custom_metric(y_true, y_scores, min_tpr=min_tpr)

    return fpr, tpr, roc_auc, normalized_pauc, y_true, y_scores


## Model Init

In [ ]:

fine_tuned_model = ConformerTinyBinary(num_classes=2)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")
fine_tuned_model = fine_tuned_model.to(device)

In [ ]:

learning_rate = 1e-4  
num_epochs = 20  
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(fine_tuned_model.parameters(), lr=learning_rate)


## Training

In [ ]:

train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

best_val_loss = float('inf')

val_aucs = []
val_paucs = []

for epoch in range(num_epochs):
    
    train_loss = train_one_epoch(fine_tuned_model, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)

    
    train_loss, train_acc = validate(fine_tuned_model, train_loader, criterion, device)
    train_accuracies.append(train_acc)

    
    val_loss, val_acc = validate(fine_tuned_model, val_loader, criterion, device)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

    
    fpr, tpr, roc_auc, partial_auc, _, _ = compute_auc_and_roc(fine_tuned_model, val_loader, device, min_tpr=0.8)

    val_aucs.append(roc_auc)
    val_paucs.append(partial_auc)

    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val AUC: {roc_auc:.4f}, Val pAUC: {partial_auc:.4f}")

    
    save_model(epoch, fine_tuned_model, optimizer, os.path.join(model_dir, f"epoch{epoch+1}.pth"))
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        save_model(epoch+1, fine_tuned_model, optimizer, os.path.join(model_dir, f"best_model_epoch{epoch+1}.pth"), best=True)

## Testing

In [ ]:
import glob
import re
import os


model_files = glob.glob(os.path.join(model_dir, "best_model_epoch*.pth"))
if not model_files:
    raise FileNotFoundError(f"No model files found in {model_dir} matching pattern 'best_model_epoch*.pth'")


epoch_numbers = [
    int(re.search(r"best_model_epoch(\d+)\.pth", os.path.basename(file)).group(1))
    for file in model_files
]


highest_epoch = max(epoch_numbers)
print(f"Highest epoch found: {highest_epoch}")


best_model_path = os.path.join(model_dir, f"best_model_epoch{highest_epoch}.pth")
print(f"Loading best model from: {best_model_path}")
checkpoint = torch.load(best_model_path)
fine_tuned_model.load_state_dict(checkpoint['model_state_dict'])


fpr, tpr, roc_auc, partial_auc, y_true, y_scores = compute_auc_and_roc(fine_tuned_model, test_loader, device, min_tpr=0.8)


y_pred = [1 if score >= 0.5 else 0 for score in y_scores]
class_report = classification_report(y_true, y_pred, target_names=dataset.classes)
print("Classification Report:")
print(class_report)


report_path = os.path.join(model_dir, "classification_report.txt")
with open(report_path, "w") as report_file:
    report_file.write(class_report)

print(f"Classification report saved at: {report_path}")


plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.4f}", linewidth=2)
plt.fill_between(fpr, tpr, alpha=0.2, label=f"pAUC = {partial_auc:.4f} (FPR ≥ 0.8)")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve (Test Set)")
plt.legend(loc="lower right")
plt.grid(True)


roc_plot_path = os.path.join(model_dir, "test_roc_curve.png")
plt.savefig(roc_plot_path, bbox_inches="tight")
plt.show()

print(f"Test ROC Curve plot saved at: {roc_plot_path}")

## Plotting

In [ ]:

plt.figure(figsize=(15, 12))


plt.subplot(3, 1, 1)
plt.plot(range(1, num_epochs + 1), train_losses, label='Train Loss', marker='o')
plt.plot(range(1, num_epochs + 1), val_losses, label='Validation Loss', marker='o')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Train vs Validation Loss")
plt.legend()
plt.grid(True)


plt.text(num_epochs - 1, val_losses[-1], f"Final Val Loss: {val_losses[-1]:.4f}", 
         fontsize=10, color='orange')


plt.subplot(3, 1, 2)
plt.plot(range(1, num_epochs + 1), train_accuracies, label='Train Accuracy', marker='o', color='blue')
plt.plot(range(1, num_epochs + 1), val_accuracies, label='Validation Accuracy', marker='o', color='orange')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Train vs Validation Accuracy")
plt.legend()
plt.grid(True)



test_accuracy = sum([y_true[i] == y_pred[i] for i in range(len(y_true))]) / len(y_true)
print(f"Test Accuracy: {test_accuracy:.4f}")
plt.text(num_epochs - 1, val_accuracies[-1], f"Test Accuracy: {test_accuracy:.4f}", 
         fontsize=10, color='red', bbox=dict(facecolor='white', alpha=0.5))


plt.subplot(3, 1, 3)
plt.plot(range(1, num_epochs + 1), val_aucs, label="Validation AUC", marker='o', color='green')
plt.plot(range(1, num_epochs + 1), val_paucs, label="Validation pAUC (FPR ≤ 0.1)", marker='o', color='purple')
plt.xlabel("Epochs")
plt.ylabel("AUC")
plt.title("Validation AUC and pAUC")
plt.legend()
plt.grid(True)


plt.text(num_epochs - 1, val_aucs[-1], f"Test AUC: {roc_auc:.4f}", 
         fontsize=10, color='red', bbox=dict(facecolor='white', alpha=0.5))
plt.text(num_epochs - 1, val_paucs[-1], f"Test pAUC: {partial_auc:.4f}", 
         fontsize=10, color='red', bbox=dict(facecolor='white', alpha=0.5))


combined_plot_path = os.path.join(model_dir, "combined_metrics.png")
plt.tight_layout()
plt.savefig(combined_plot_path, bbox_inches="tight")
plt.show()

print(f"Combined metrics plot saved at: {combined_plot_path}")
